In [23]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier

from sklearn.datasets import make_classification
from sklearn.metrics import f1_score

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

import xgboost as xgb

### Load BRCA data

In [2]:
exp_file = "./data/exp/tcga_brca_mRNAseq_RSEM_normalized_log2_plus1_preprocessed.pkl"
exp_df = pd.read_pickle(exp_file)
exp_df

,TCGA-3C-AAAU-01,TCGA-3C-AALI-01,TCGA-3C-AALJ-01,TCGA-3C-AALK-01,TCGA-4H-AAAK-01,TCGA-5L-AAT0-01,TCGA-5L-AAT1-01,TCGA-5T-A9QA-01,TCGA-A1-A0SB-01,TCGA-A1-A0SD-01,...,TCGA-UL-AAZ6-01,TCGA-UU-A93S-01,TCGA-V7-A7HQ-01,TCGA-W8-A86G-01,TCGA-WT-AB41-01,TCGA-WT-AB44-01,TCGA-XX-A899-01,TCGA-XX-A89A-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01
A1BG,7.630010,7.897146,8.728725,7.585096,8.076179,7.677873,7.729932,8.300051,5.649592,7.162871,...,5.496785,8.335280,10.012705,9.060797,6.916604,8.883514,7.808774,8.000758,8.783137,7.961896
A1CF,0.000000,0.000000,0.931002,0.000000,0.511468,0.000000,0.000000,0.606916,0.000000,0.000000,...,0.484602,0.000000,0.000000,0.560324,0.000000,0.000000,0.000000,1.749234,0.000000,0.000000
A2BP1,0.000000,0.000000,0.000000,0.000000,2.271963,0.665938,0.000000,0.606916,2.460743,0.412294,...,0.000000,0.000000,0.692249,2.637077,2.519013,0.000000,0.478920,1.273277,0.675635,0.000000
A2LD1,6.699932,6.167209,7.342165,5.988848,7.279566,6.814235,6.686112,7.874177,5.281557,6.084553,...,7.568796,7.820780,6.979354,5.923986,8.290288,5.971102,7.046887,6.338587,6.362838,4.710757
A2ML1,1.250113,2.419593,0.000000,1.408658,2.138913,1.119821,2.343408,0.000000,2.054328,3.105846,...,7.177123,0.000000,4.088464,1.277271,0.000000,2.909965,0.478920,5.826276,1.760987,2.371754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYX,11.776535,12.426690,12.414619,12.474809,11.981104,12.012968,12.109535,11.654080,12.595195,11.797934,...,12.023927,12.449940,12.656059,12.438674,12.704956,13.417072,12.321677,12.419582,12.704904,11.732813
ZZEF1,10.888693,10.365941,9.882464,9.611271,9.702025,9.850389,10.464110,10.124202,10.916102,10.321892,...,10.280054,9.075079,9.548103,9.923476,9.127632,9.646832,10.918012,10.490261,9.561121,9.143177
ZZZ3,10.206351,8.671516,8.995822,9.455058,9.785783,10.053572,9.154482,8.787575,10.489043,10.224725,...,8.283531,8.597667,8.250810,9.768030,8.170257,8.794698,9.390338,9.898916,9.554260,8.855224
psiTPTE22,1.445356,9.857344,5.184205,6.079197,7.556383,6.427212,5.879331,2.220670,9.111956,6.443182,...,3.558145,3.041926,8.169826,7.789327,4.877312,9.503634,6.635540,7.884100,7.906453,4.451449


### Load BRCA label

In [24]:
label_file = "./data/exp/tcga_brca_pam50_refine.csv"
label_mapping = {"LumA":0 , "LumB":1 , "Her2":2, "Basal":3, "Normal":4}
label_df = pd.read_csv("./data/exp/tcga_brca_pam50_refine.csv",index_col=0)
label_df['Subtype_index'] = list(map(lambda x:label_mapping[x],label_df['Subtype']))
label_dict = dict(zip(label_df.index, label_df['Subtype_index']))

### Load Node genes

In [31]:
node_gene_file = "./data/preprocessed/tcga_brca_gene_mapping.pkl"
f = open(node_gene_file,'rb')
node_genes = list(pickle.load(f).keys())
f.close()

In [32]:
len(node_genes)

9630

### Data preparation

In [9]:
X = exp_df.T
X = X.sample(frac=1)
#X = X.loc[:, node_genes]

In [10]:
y = list(map(lambda x:label_dict[x], X.index))

In [11]:
X.shape

(1212, 9630)

### Cross validation

In [12]:
model = RandomForestClassifier(n_estimators=500)
#model = OneVsRestClassifier(SVC(random_state=0))
#model = xgb.XGBClassifier()

In [8]:
y_pred = cross_val_predict(model, X, y, cv=10,n_jobs=4)

In [9]:
f1_score(y, y_pred, average='weighted')

0.885983677286554

In [10]:
X.shape

(1212, 20502)

### Load pre-CV dataset 

In [25]:
baseline_dataset_file = "./data/preprocessed/tcga_brca_baseline_dataset.pkl"
f = open(baseline_dataset_file,'rb')
baseline_dataset = pickle.load(f)
f.close()

exp_df = baseline_dataset['exp_data']
train_index_list = baseline_dataset['train_index']
test_index_list = baseline_dataset['test_index']
label_dict = baseline_dataset['labels']

In [26]:
f1_list = []
for i in [0,1,2,3,4,5,6,7,8,9]:
    train_index, test_index = train_index_list[i], test_index_list[i]
    X_train,X_test = exp_df.iloc[:,train_index], exp_df.iloc[:,test_index]
    y_train = list(map(lambda x:label_dict[x], X_train.columns.tolist()))
    y_test = list(map(lambda x:label_dict[x], X_test.columns.tolist()))

    X_train,X_test = X_train.T, X_test.T
    #X_train = X_train.sample(frac=1)

    model = xgb.XGBClassifier()
    model.fit(X_train.values, y_train)
    y_pred = model.predict(X_test.values)
    f1_list.append(f1_score(y_test, y_pred, average="weighted"))
                   
    print(i,":",f1_score(y_test, y_pred, average='weighted'))
                   
print(sum(f1_list)/len(f1_list))

0 : 0.8697179164228643
1 : 0.9345027994726011
2 : 0.9336631747698941
3 : 0.9753015429583931
4 : 0.9249755668772454
5 : 0.9158950511249693
6 : 0.9075074628928383
7 : 0.9177985083381487
8 : 0.893100564576516
9 : 0.9258080590342894
0.9198270646467762


In [13]:
#pagerank
baseline_dataset_file = "./data/preprocessed/tcga_brca_baseline_dataset_pagerank.pkl"
f = open(baseline_dataset_file,'rb')
baseline_dataset = pickle.load(f)
f.close()

exp_df = baseline_dataset['exp_data']
train_index_list = baseline_dataset['train_index']
test_index_list = baseline_dataset['test_index']
label_dict = baseline_dataset['labels']

In [14]:
#pagerank
f1_list = []
for i in [0,1,2,3,4,5,6,7,8,9]:
    train_index, test_index = train_index_list[i], test_index_list[i]
    X_train,X_test = exp_df.iloc[:,train_index], exp_df.iloc[:,test_index]
    y_train = list(map(lambda x:label_dict[x], X_train.columns.tolist()))
    y_test = list(map(lambda x:label_dict[x], X_test.columns.tolist()))

    X_train,X_test = X_train.T, X_test.T
    #X_train = X_train.sample(frac=1)

    model = xgb.XGBClassifier()
    model.fit(X_train.values, y_train)
    y_pred = model.predict(X_test.values)
    f1_list.append(f1_score(y_test, y_pred, average="weighted"))
                   
    print(i,":",f1_score(y_test, y_pred, average='weighted'))
                   
print(sum(f1_list)/len(f1_list))

/home/minjae/miniconda3/envs/minjae/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0 : 0.8199708454810496
1 : 0.9492882823391298
2 : 0.861748644158124
3 : 0.8979591836734694
4 : 0.8843257908846409
5 : 0.8995506205842416
6 : 0.8765460729746443
7 : 0.8668982288745983
8 : 0.8942482083428713
9 : 0.925784008979956
0.8876319886292725


In [36]:
#ori_netics
baseline_dataset_file = "./data/preprocessed/tcga_brca_baseline_dataset_ori_exp.pkl"
f = open(baseline_dataset_file,'rb')
baseline_dataset = pickle.load(f)
f.close()

exp_df = baseline_dataset['exp_data']
train_index_list = baseline_dataset['train_index']
test_index_list = baseline_dataset['test_index']
label_dict = baseline_dataset['labels']

In [37]:
#ori_netics
f1_list = []
for i in [0,1,2,3,4,5,6,7,8,9]:
    train_index, test_index = train_index_list[i], test_index_list[i]
    X_train,X_test = exp_df.iloc[:,train_index], exp_df.iloc[:,test_index]
    y_train = list(map(lambda x:label_dict[x], X_train.columns.tolist()))
    y_test = list(map(lambda x:label_dict[x], X_test.columns.tolist()))

    X_train,X_test = X_train.T, X_test.T
    #X_train = X_train.sample(frac=1)

    model = xgb.XGBClassifier()
    model.fit(X_train.values, y_train)
    y_pred = model.predict(X_test.values)
    f1_list.append(f1_score(y_test, y_pred, average="weighted"))
                   
    print(i,":",f1_score(y_test, y_pred, average='weighted'))
                   
print(sum(f1_list)/len(f1_list))

0 : 0.8335352545237598
1 : 0.9590506345954289
2 : 0.8765914643883458
3 : 0.8661373363935241
4 : 0.9243593467195952
5 : 0.8899296560729555
6 : 0.8863842506699648
7 : 0.8772446091855627
8 : 0.8945578231292517
9 : 0.8953177502058357
0.8903108125884224


### Feature importance 분석

In [27]:
importance = pd.Series(model.feature_importances_,index=X_train.columns)

In [28]:
importance

0       0.001021
1       0.000266
2       0.000616
3       0.000791
4       0.000000
          ...   
9625    0.000000
9626    0.000000
9627    0.000000
9628    0.000000
9629    0.000000
Length: 9630, dtype: float32

In [29]:
indexlist=importance.sort_values(ascending=False)[0:100].index

In [34]:
importance = []
for i in indexlist:
    importance.append(node_genes[i])
importance

['LMOD1',
 'SFRP1',
 'NCAPG',
 'ECE2',
 'SPRY2',
 'GATA3',
 'MLPH',
 'ERBB2',
 'CFB',
 'ACE2',
 'WWP1',
 'TBC1D9',
 'PLK1',
 'DMD',
 'ADCYAP1R1',
 'UHRF1',
 'KPNA4',
 'COL10A1',
 'CHST3',
 'TPX2',
 'PTPRF',
 'CENPA',
 'USP44',
 'HOXA4',
 'CEP55',
 'KIF4A',
 'KRT5',
 'NSL1',
 'CXCL13',
 'ACTG2',
 'ACTA2',
 'MYL7',
 'FOXA1',
 'GALT',
 'G6PD',
 'LYPLA1',
 'ESR1',
 'ANXA8',
 'ELOVL1',
 'EIF2AK2',
 'SCN4B',
 'CCNA2',
 'SLC9A3R1',
 'GTSE1',
 'HIST1H2BD',
 'AURKA',
 'SERPINB5',
 'PRODH',
 'ANGPT1',
 'SLC7A5',
 'SOSTDC1',
 'TRIM29',
 'ACSL4',
 'ASAP3',
 'STBD1',
 'C10orf95',
 'BIRC5',
 'FOLR2',
 'KLHL7',
 'PLAU',
 'KIFC1',
 'SLC25A27',
 'GPR3',
 'DNAJC1',
 'CDC45',
 'NEFM',
 'LZTS1',
 'ANLN',
 'RGS20',
 'APOC1',
 'HPN',
 'SLC22A5',
 'CDC25C',
 'GPR182',
 'EXOSC7',
 'NEU1',
 'KRT14',
 'GGCX',
 'STC2',
 'IRAK3',
 'FOXC1',
 'EZH1',
 'USP36',
 'GRB7',
 'ITGA3',
 'AQP4',
 'CDKN3',
 'MIR17HG',
 'TRAPPC9',
 'UBE2C',
 'PSAT1',
 'SKA1',
 'SERPINA3',
 'CLDN19',
 'DSC3',
 'SERPINA6',
 'MMACHC',
 'TOX4',


In [15]:
importance = pd.Series(model.feature_importances_,index=X_train.columns)

In [16]:
importance

0       0.000000
1       0.002228
2       0.000000
3       0.000000
4       0.000000
          ...   
6011    0.000000
6012    0.000000
6013    0.000000
6014    0.000000
6015    0.000000
Length: 6016, dtype: float32

In [19]:
indexlist=importance.sort_values(ascending=False)[0:100].index

In [18]:
#pagerank
with open('./data/preprocessed/netics_node_mapping.pkl', 'rb') as f:
    mapping = pickle.load(f)

In [21]:
importance=[]
for i in indexlist:
    importance.append(list(mapping.items())[i])

In [22]:
importance

[('AURKA', 358),
 ('FOXA1', 1783),
 ('DST', 1396),
 ('CENPA', 814),
 ('SFRP1', 5001),
 ('CCNB1', 647),
 ('CDT1', 801),
 ('UBB', 5757),
 ('MET', 3037),
 ('CHRNA6', 897),
 ('ERBB2', 1577),
 ('CCNA2', 646),
 ('ITPR2', 2572),
 ('LBP', 2775),
 ('EGFR', 1465),
 ('CCL21', 632),
 ('GATA3', 1897),
 ('CELSR3', 813),
 ('RACGAP1', 4578),
 ('ATF6', 308),
 ('TFAP2A', 5495),
 ('NDN', 3312),
 ('APEX1', 186),
 ('ST3GAL1', 5276),
 ('PRC1', 4367),
 ('ESR1', 1597),
 ('KPNA2', 2750),
 ('VEGFC', 5871),
 ('NDC80', 3309),
 ('FOXM1', 1788),
 ('CDC25B', 737),
 ('IGF2', 2390),
 ('FANCD2', 1666),
 ('BCL6', 413),
 ('MYH11', 3233),
 ('IRAK2', 2519),
 ('MRPL46', 3136),
 ('MC2R', 2975),
 ('XBP1', 5946),
 ('PTGS2', 4503),
 ('CCNB2', 648),
 ('CRMP1', 1091),
 ('CXCL12', 1166),
 ('KIR3DL1', 2709),
 ('BCL2L2', 411),
 ('NCAPH', 3283),
 ('PRKCH', 4398),
 ('NEFM', 3323),
 ('EDNRB', 1442),
 ('LTBP4', 2874),
 ('TP63', 5641),
 ('NGFR', 3359),
 ('CHMP4C', 880),
 ('PPBP', 4307),
 ('WWC1', 5941),
 ('CDK2', 780),
 ('TNFRSF10A', 558

In [38]:
#ori_netics
importance = pd.Series(model.feature_importances_,index=X_train.columns)

In [39]:
importance

0       0.000000
1       0.001734
2       0.000315
3       0.000000
4       0.000000
          ...   
6011    0.000000
6012    0.000000
6013    0.000142
6014    0.000000
6015    0.000000
Length: 6016, dtype: float32

In [40]:
indexlist=importance.sort_values(ascending=False)[0:100].index

In [41]:
importance=[]
for i in indexlist:
    importance.append(list(mapping.items())[i])

In [42]:
importance

[('SFRP1', 5001),
 ('NCAPG', 3281),
 ('FOXA1', 1783),
 ('ERBB2', 1577),
 ('FANCA', 1663),
 ('IL1B', 2446),
 ('DUSP7', 1403),
 ('GATA3', 1897),
 ('CCNA2', 646),
 ('TPX2', 5649),
 ('DST', 1396),
 ('TGFBR3', 5517),
 ('ZP2', 6007),
 ('MSI2', 3184),
 ('PAK7', 3993),
 ('CCR7', 669),
 ('LMOD1', 2829),
 ('PTPRF', 4525),
 ('NUP85', 3517),
 ('CISH', 912),
 ('PPP1R1B', 4333),
 ('NOS1AP', 3397),
 ('FANCB', 1664),
 ('NCAPH', 3283),
 ('ESR1', 1597),
 ('SF3A3', 4992),
 ('BIRC5', 434),
 ('DFFA', 1244),
 ('UNC5A', 5794),
 ('NEFM', 3323),
 ('DMD', 1343),
 ('NEFL', 3322),
 ('SCUBE2', 4927),
 ('SGK2', 5009),
 ('RUVBL1', 4869),
 ('KIF4A', 2692),
 ('TUBGCP2', 5732),
 ('WNT5A', 5929),
 ('COL17A1', 994),
 ('CLDN2', 933),
 ('PARN', 4006),
 ('MYH9', 3244),
 ('SAV1', 4908),
 ('BDNF', 422),
 ('GPR17', 2024),
 ('GRB7', 2043),
 ('MAG', 2895),
 ('PABPC1L', 3975),
 ('WWP1', 5943),
 ('ARPC2', 264),
 ('KCNJ12', 2638),
 ('PARD3', 4000),
 ('SGOL1', 5011),
 ('SKA1', 5058),
 ('AKT1', 125),
 ('GABRP', 1859),
 ('GATAD2B', 19